In [1]:
%pip install torchattacks
%pip install pytorchcv
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image

import torchattacks

from tqdm import tqdm
import math
import random
import csv
import torchvision.transforms as transforms
from PIL import Image

## Transforms

In [2]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    transforms.Resize((64, 64)),
    # Resize the image into a fixed shape (height = width = 128)
    # You may add some transforms here.
#     transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
#     transforms.RandomRotation(degrees=(0, 180)), 
#     transforms.RandomHorizontalFlip(p=0.5), 
#     transforms.RandomVerticalFlip(p=0.5),
#     transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5, saturation=0.5),
    # transforms.RandomChoice(transform_set_color),
    # transforms.RandomChoice(transform_set_pad),
    # transforms.Resize((128, 128)),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

## Useful Functions

### set seeds

In [3]:
def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Datasets class

In [4]:
class Cifar100(Dataset):
    def __init__(self, data, labels):
        super(Cifar100, self).__init__()
        self.data = data
        self.labels = labels
        self.datasize = len(self.data)

    def __getitem__(self, idx):
        self.data[idx]
        return self.data[idx], self.labels[idx]
        
            
    def __len__(self):
        return self.datasize

## HyperParameters

In [14]:
config = {
    "batch_size": 1,
    "model_type": "single",
    "attack": "MIFGSM",
    "model_name": "resnet164bn_cifar100",
    "model_names": ['nin_cifar100', 'resnet20_cifar100', 'preresnet20_cifar100', 'seresnet20_cifar100', 'sepreresnet20_cifar100', 'pyramidnet110_a48_cifar100', 'densenet40_k12_cifar100', 'xdensenet40_2_k24_bc_cifar100', 'wrn16_10_cifar100', 'wrn20_10_1bit_cifar100', 'ror3_56_cifar100', 'rir_cifar100', 'resnext29_16x64d_cifar100', 'diaresnet20_cifar100', 'diapreresnet20_cifar100'],
    "test_model_names": ['resnet20_cifar100', 'wrn16_10_cifar100', 'ror3_164_cifar100', 'rir_cifar100', 'pyramidnet110_a270_cifar100', 'resnet110_cifar100', "preresnet164bn_cifar100", "resnext29_32x4d_cifar100", "seresnet164bn_cifar100", 'sepreresnet272bn_cifar100', 'densenet100_k12_cifar100', 'diaresnet164bn_cifar100', 'diapreresnet164bn_cifar100'],
    "data_filepath": "./cifar-100_eval",
    "current_dir": "./",
    "save_dir": "adv_imgs",
    "output_filepath": "./adv_imgs",
    "steps": 16,
    "epsilon": 8 / 255,
    "alpha" : 0.5 / 255,
    "decay": 0.9, 
    "seeds": 10901036
}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load datasets

In [15]:
same_seeds(config["seeds"])
eval_files = sorted([os.path.join(config["data_filepath"],x) for x in os.listdir(config["data_filepath"]) if x.endswith(".png")])
eval_imgs = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in eval_files])
eval_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in eval_files], dtype=torch.long)

## Model(s)

### Single model

In [16]:
from pytorchcv.model_provider import get_model as ptcv_get_model
import torchattacks
adv_images = None
if config["model_type"] == "single":
    model_name = config["model_name"]
    model = ptcv_get_model(model_name, pretrained=True).to(device)
    attack = None
    if config["attack"] == "FGSM":
        attack = torchattacks.FGSM(model, eps=config["epsilon"])
    elif config["attack"] == "MIFGSM":
        attack = torchattacks.MIFGSM(model, eps=config["epsilon"], decay=config["decay"], steps=config["steps"])
    elif config["attack"] == "PGD":
        attack = torchattacks.PGD(model, eps=config["epsilon"], alpha=config["alpha"], steps=config["steps"])
    adv_images = attack(eval_imgs, eval_labels)
    with open("attack_info.txt", 'w') as f:
        if config["attack"] == "FGSM":
            f.write(f"Attack: FGSM\nEpsilon: {config['epsilon']}\nModel_type: {config["model_type"]}\nModel: {model_name}\n")
        elif config["attack"] == "PGD":
            f.write(f"Attack: PGD \nEpsilon: {config['epsilon']}\nAlpha: {config["alpha"]}\nSteps: {config["steps"]}\nModel_type: {config["model_type"]}\nModel: {model_name}\n")
        elif config["attack"] == "MIFGSM":
            f.write(f"Attack: MIFGSM \nEpsilon: {config['epsilon']}\nDecay: {config["decay"]}\nSteps: {config["steps"]}\nModel_type: {config["model_type"]}\nModel: {model_name}\n")

### Multiple models

In [17]:
if config["model_type"] == "ensemble":
    model_names = config["model_names"]
    models = [ptcv_get_model(model_name, pretrained=True).to(device) for model_name in model_names]
    with open("attack_info.txt", 'w') as f:
        if config["attack"] == "FGSM":
            f.write(f"Attack: FGSM\nEpsilon: {config['epsilon']}\nModel_type: {config["model_type"]}\nModel: {model_names}\n")
        elif config["attack"] == "PGD":
            f.write(f"Attack: PGD \nEpsilon: {config['epsilon']}\nModel_type: {config["model_type"]}\nAlpha: {config["alpha"]}\nSteps: {config["steps"]}\nModel_type: {config["model_type"]}\nModel: {model_names}\n")
        elif config["attack"] == "MIFGSM":
            f.write(f"Attack: MIFGSM \nEpsilon: {config['epsilon']}\nModel_type: {config["model_type"]}\nDecay: {config["decay"]}\nSteps: {config["steps"]}\nModel_type: {config["model_type"]}\nModel: {model_names}\n")

    final_adv_images = 0
    models_num = len(models)
    for i, model in tqdm(enumerate(models)):
        if config["attack"] == "FGSM":
            attack = torchattacks.FGSM(model, eps=config["epsilon"])
        elif config["attack"] == "MIFGSM":
            attack = torchattacks.MIFGSM(model, eps=config["epsilon"], decay=config["decay"], steps=config["steps"])
        elif config["attack"] == "PGD":
            attack = torchattacks.PGD(model, eps=config["epsilon"], alpha=config["alpha"], steps=config["steps"])
        adv_images = attack(eval_imgs, eval_labels)
        if i==0: 
            final_adv_images = adv_images / models_num
        else:
            final_adv_images += adv_images / models_num

    adv_images = final_adv_images  

if adv_images == None:
    print("No model found.")
    exit()

## Save images

In [18]:
if not os.path.exists(config["save_dir"]):
    os.mkdir(config["save_dir"])
for adv_image, eval_file in zip(adv_images, eval_files):
    save_image(adv_image, os.path.join(config["current_dir"], config["save_dir"], eval_file.split('/')[-1]))

## Test Accuracies

In [19]:
adv_files = sorted([os.path.join(config["output_filepath"], x) for x in os.listdir(config["output_filepath"]) if x.endswith(".png")])
adv_images = torch.stack([transforms.ToTensor()(Image.open(filename)) for filename in adv_files])
adv_labels = torch.tensor([int(filename.split('/')[-1].split('_')[0]) for filename in adv_files], dtype=torch.long)

eval_datasets = Cifar100(eval_imgs, eval_labels)
adv_datasets  = Cifar100(adv_images, adv_labels)

eval_loader = DataLoader(eval_datasets, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
adv_loader  = DataLoader(adv_datasets , batch_size=config['batch_size'], shuffle=False, pin_memory=True)

In [20]:
test_model_names = config["test_model_names"]
test_models = [ptcv_get_model(test_model_name, pretrained=True).to(device) for test_model_name in test_model_names]

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    acc = 0
    for img, label in tqdm(test_loader):
        img = img.to(device)
        label = label.to(device)
        with torch.no_grad():                   
            pred = model(img) 
            if label[0] == torch.argmax(pred):
                acc += 1
    return acc / 500

for idx, test_model in enumerate(test_models):
    original_acc = predict(eval_loader, test_model, device)
    adv_acc = predict(adv_loader, test_model, device)
    print(f"original images accuracy: {original_acc}")
    print(f"adversarial images accuracy: {adv_acc}")
    print("")
    with open("attack_info.txt", 'a') as f:
        f.write(f"\nTestModel: {test_model_names[idx]}\nOriginal images accuracy: {original_acc}\nAdversarial images accuracy: {adv_acc}\n")

100%|██████████| 500/500 [00:01<00:00, 322.34it/s]


original images accuracy: 0.496
adversarial images accuracy: 0.104



100%|██████████| 500/500 [00:25<00:00, 19.64it/s]


original images accuracy: 0.978
adversarial images accuracy: 0.15



100%|██████████| 500/500 [00:13<00:00, 36.68it/s]


...

original images accuracy: 0.936
adversarial images accuracy: 0.114



100%|██████████| 500/500 [00:27<00:00, 18.02it/s]


original images accuracy: 0.848
adversarial images accuracy: 0.114



100%|██████████| 500/500 [00:26<00:00, 18.52it/s]

original images accuracy: 0.9
adversarial images accuracy: 0.114



In [21]:
if config["model_type"] == "single":
    os.system(f"zip -r {config['model_name']+"_"+config["attack"]}.zip adv_imgs attack_info.txt")
else:
    model_names = "_".join(config["model_names"])
    os.system(f"zip -r {model_names+"_"+config["attack"]}_step{config["steps"]}.zip adv_imgs attack_info.txt")

updating: adv_imgs/ (stored 0%)
updating: adv_imgs/29_2.png (stored 0%)
updating: adv_imgs/91_2.png (stored 0%)
updating: adv_imgs/12_4.png (stored 0%)
updating: adv_imgs/57_4.png (stored 0%)
updating: adv_imgs/48_3.png (stored 0%)
updating: adv_imgs/93_0.png (stored 0%)
updating: adv_imgs/88_3.png (stored 0%)
updating: adv_imgs/53_0.png (stored 0%)
updating: adv_imgs/16_0.png (stored 0%)
updating: adv_imgs/30_3.png (stored 0%)
updating: adv_imgs/75_3.png (stored 0%)
updating: adv_imgs/97_4.png (stored 0%)
updating: adv_imgs/8_0.png (stored 0%)
updating: adv_imgs/14_2.png (stored 0%)
updating: adv_imgs/51_2.png (stored 0%)
updating: adv_imgs/77_1.png (stored 0%)
updating: adv_imgs/32_1.png (stored 0%)
updating: adv_imgs/77_0.png (stored 0%)
updating: adv_imgs/32_0.png (stored 0%)
updating: adv_imgs/14_3.png (stored 0%)
updating: adv_imgs/51_3.png (stored 0%)
updating: adv_imgs/8_1.png (stored 0%)
updating: adv_imgs/30_2.png (stored 0%)
updating: adv_imgs/75_2.png (stored 0%)
updating: 